# 데이터 불러오기

In [167]:
import pandas as pd
import os

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)

PATH = './data/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['rent', 'living_popul', 'resident_popul', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    if name == 'rent':
        globals()[name] = pd.read_csv(PATH + file, names = ['기준년코드', '기준분기코드', '행정동코드', '행정동코드명', '임대시세층구분코드', '임대시세층구분명', '보증금평균','월임대료평균','환산임대료평균','임대건수'],encoding = "ansi")
    else:
        globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

# 데이터 전처리

## 시점 통일

In [168]:
years = [20223, 20224, 20231, 20232]

living_popul = living_popul[living_popul['기준_년분기_코드'].isin(years)]
resident_popul = resident_popul[resident_popul['기준_년분기_코드'].isin(years)]
store = store[store['기준_년분기_코드'].isin(years)]
working_popul = working_popul[working_popul['기준_년분기_코드'].isin(years)]
facilitie = facilitie[facilitie['기준_년분기_코드'].isin(years)]
estimated_sales = estimated_sales[estimated_sales['기준_년분기_코드'].isin(years)]

## 길단위인구 전처리

In [169]:
living_popul['시간대_21_06_유동인구_수'] = living_popul['시간대_00_06_유동인구_수'] + living_popul['시간대_21_24_유동인구_수']
living_popul['주중_유동인구_수'] = living_popul['월요일_유동인구_수'] + living_popul['화요일_유동인구_수'] + living_popul['수요일_유동인구_수'] + living_popul['목요일_유동인구_수'] + living_popul['금요일_유동인구_수']
living_popul['주말_유동인구_수'] = living_popul['토요일_유동인구_수'] + living_popul['일요일_유동인구_수']
living_popul = living_popul.drop(['월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수', '시간대_00_06_유동인구_수', '시간대_21_24_유동인구_수'], axis = 1)

living_popul_year = living_popul.groupby(['상권_구분_코드_명','상권_코드_명']).mean()
living_popul_year = living_popul_year.reset_index()
living_popul_year = living_popul_year.drop('기준_년분기_코드', axis = 1)

In [170]:
living_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권_구분_코드_명        1650 non-null   object 
 1   상권_코드_명           1650 non-null   object 
 2   총_유동인구_수          1650 non-null   float64
 3   남성_유동인구_수         1650 non-null   float64
 4   여성_유동인구_수         1650 non-null   float64
 5   연령대_10_유동인구_수     1650 non-null   float64
 6   연령대_20_유동인구_수     1650 non-null   float64
 7   연령대_30_유동인구_수     1650 non-null   float64
 8   연령대_40_유동인구_수     1650 non-null   float64
 9   연령대_50_유동인구_수     1650 non-null   float64
 10  연령대_60_이상_유동인구_수  1650 non-null   float64
 11  시간대_06_11_유동인구_수  1650 non-null   float64
 12  시간대_11_14_유동인구_수  1650 non-null   float64
 13  시간대_14_17_유동인구_수  1650 non-null   float64
 14  시간대_17_21_유동인구_수  1650 non-null   float64
 15  시간대_21_06_유동인구_수  1650 non-null   float64
 16  주중_유동인구_수         1650 non-null   float64


In [171]:
living_popul_year.describe()

,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수
count,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000,1650.000
mean,831346.164,396274.407,435071.771,106200.819,148784.816,147328.849,134100.080,121060.744,173870.929,169119.867,107349.673,108418.850,144230.136,302227.713,599837.928,231508.316
std,899909.503,437201.237,466297.269,113067.081,216245.860,182823.763,151684.683,128492.073,182497.671,179602.502,134019.074,136916.252,167242.310,320730.232,660145.648,245728.354
min,249.000,177.000,72.000,0.000,6.000,19.667,40.667,34.000,47.667,68.333,44.000,37.333,39.667,59.667,194.000,55.000
25%,224722.000,106555.125,115045.188,25827.562,32070.500,36495.500,36211.500,33183.938,46118.438,46207.875,28653.312,29065.562,38389.062,75899.875,160303.750,60267.312
50%,569974.625,270697.000,297988.000,72271.500,82953.500,93360.625,91485.250,83814.125,119785.875,117905.500,71894.500,72122.000,96428.000,206994.750,407303.375,160281.000
75%,1140857.500,541802.188,593083.000,147602.062,184640.375,191001.938,179951.375,169102.625,240064.125,235774.750,139710.375,140308.438,194904.375,413306.750,828801.375,311599.375
max,8536354.500,4598446.750,3937907.750,804173.750,3262037.500,1984148.500,1626110.750,1521976.000,1958301.500,1648813.000,1805141.250,1830343.500,1719183.000,2680625.000,6772121.500,2029885.000


## 상주인구 전처리

In [172]:
resident_popul.drop(['총_상주인구_수','남성_상주인구_수','여성_상주인구_수','연령대_10_상주인구_수','연령대_20_상주인구_수','연령대_30_상주인구_수','연령대_40_상주인구_수','연령대_50_상주인구_수','연령대_60_이상_상주인구_수','총_가구_수','아파트_가구_수','비_아파트_가구_수'],axis=1,inplace=True)

resident_popul_year = resident_popul[resident_popul['기준_년분기_코드'] == 20232]
resident_popul_year.drop(['기준_년분기_코드'],axis=1,inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16068\4201384229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resident_popul_year.drop(['기준_년분기_코드'],axis=1,inplace=True)


In [173]:
resident_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1625 entries, 19887 to 24485
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   상권_구분_코드_명          1625 non-null   object
 1   상권_코드_명             1625 non-null   object
 2   남성연령대_10_상주인구_수     1625 non-null   int64 
 3   남성연령대_20_상주인구_수     1625 non-null   int64 
 4   남성연령대_30_상주인구_수     1625 non-null   int64 
 5   남성연령대_40_상주인구_수     1625 non-null   int64 
 6   남성연령대_50_상주인구_수     1625 non-null   int64 
 7   남성연령대_60_이상_상주인구_수  1625 non-null   int64 
 8   여성연령대_10_상주인구_수     1625 non-null   int64 
 9   여성연령대_20_상주인구_수     1625 non-null   int64 
 10  여성연령대_30_상주인구_수     1625 non-null   int64 
 11  여성연령대_40_상주인구_수     1625 non-null   int64 
 12  여성연령대_50_상주인구_수     1625 non-null   int64 
 13  여성연령대_60_이상_상주인구_수  1625 non-null   int64 
dtypes: int64(12), object(2)
memory usage: 190.4+ KB


In [174]:
resident_popul_year.describe()

,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수
count,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000,1625.000
mean,114.767,129.260,211.772,198.903,189.116,324.342,108.581,127.023,212.106,183.610,185.248,373.239
std,131.209,136.889,234.119,212.506,189.605,318.896,124.899,131.433,237.426,198.658,191.162,372.583
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,25.000,31.000,53.000,51.000,51.000,89.000,25.000,32.000,51.000,45.000,47.000,99.000
50%,73.000,86.000,140.000,130.000,130.000,221.000,69.000,85.000,140.000,120.000,124.000,261.000
75%,158.000,180.000,292.000,278.000,266.000,457.000,151.000,181.000,291.000,255.000,264.000,529.000
max,1586.000,1133.000,2148.000,2235.000,1577.000,2260.000,1492.000,1162.000,2070.000,2045.000,1584.000,2921.000


## 점포 전처리

In [175]:
store_year = store[store['기준_년분기_코드'] == 20232]
store_year = store_year.drop(columns = ['기준_년분기_코드'])

In [176]:
store_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76099 entries, 1284171 to 1360269
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   상권_구분_코드_명   76099 non-null  object
 1   상권_코드_명      76099 non-null  object
 2   서비스_업종_코드_명  76099 non-null  object
 3   점포_수         76099 non-null  int64 
 4   유사_업종_점포_수   76099 non-null  int64 
 5   개업_율         76099 non-null  int64 
 6   개업_점포_수      76099 non-null  int64 
 7   폐업_률         76099 non-null  int64 
 8   폐업_점포_수      76099 non-null  int64 
 9   프랜차이즈_점포_수   76099 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 6.4+ MB


In [177]:
store_year.describe()

,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
count,76099.000,76099.000,76099.000,76099.000,76099.000,76099.000,76099.000
mean,6.019,6.464,2.503,0.153,2.367,0.168,0.445
std,41.496,41.876,11.382,0.667,11.330,0.899,2.220
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1.000,1.000,0.000,0.000,0.000,0.000,0.000
50%,2.000,2.000,0.000,0.000,0.000,0.000,0.000
75%,4.000,5.000,0.000,0.000,0.000,0.000,0.000
max,8493.000,8509.000,200.000,23.000,200.000,146.000,127.000


## 직장인구 전처리

In [179]:
working_popul = working_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '총_직장_인구_수']]

working_popul_year = working_popul[working_popul['기준_년분기_코드'] == 20232]
working_popul_year = working_popul_year.drop(['기준_년분기_코드'],axis=1)

In [180]:
working_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1623 entries, 11546 to 25988
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권_구분_코드_명  1623 non-null   object
 1   상권_코드_명     1623 non-null   object
 2   총_직장_인구_수   1623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 50.7+ KB


In [181]:
working_popul_year.describe()

,총_직장_인구_수
count,1623.000
mean,1771.425
std,6247.654
min,1.000
25%,107.000
50%,317.000
75%,891.500
max,104830.000


## 집객시설 전처리

In [182]:
facilitie = facilitie.drop('집객시설_수', axis = 1)
facilitie = facilitie.fillna(0)

facilitie_year = facilitie[facilitie['기준_년분기_코드'] == 20232]
facilitie_year = facilitie_year.drop('기준_년분기_코드', axis = 1)

In [183]:
facilitie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6312 entries, 0 to 17357
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   기준_년분기_코드   6312 non-null   int64  
 1   상권_구분_코드_명  6312 non-null   object 
 2   상권_코드_명     6312 non-null   object 
 3   관공서_수       6312 non-null   float64
 4   은행_수        6312 non-null   float64
 5   종합병원_수      6312 non-null   float64
 6   일반_병원_수     6312 non-null   float64
 7   약국_수        6312 non-null   float64
 8   유치원_수       6312 non-null   float64
 9   초등학교_수      6312 non-null   float64
 10  중학교_수       6312 non-null   float64
 11  고등학교_수      6312 non-null   float64
 12  대학교_수       6312 non-null   float64
 13  백화점_수       6312 non-null   float64
 14  슈퍼마켓_수      6312 non-null   float64
 15  극장_수        6312 non-null   float64
 16  숙박_시설_수     6312 non-null   float64
 17  공항_수        6312 non-null   float64
 18  철도_역_수      6312 non-null   float64
 19  버스_터미널_수    6312 non-null   flo

## 추정매출 전처리

In [184]:
estimated_sales = estimated_sales.drop(columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액',
                                                  '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수'],
                                       axis = 1)



estimated_sales['시간대_21~06_매출_금액'] = estimated_sales['시간대_00~06_매출_금액'] + estimated_sales['시간대_21~24_매출_금액']
estimated_sales['시간대_21~06_매출_건수'] = estimated_sales['시간대_건수~06_매출_건수'] + estimated_sales['시간대_건수~24_매출_건수']

# 건당 매출 금액 컬럼 생성
estimated_sales['주말_건당_매출_금액'] = estimated_sales[(estimated_sales['주말_매출_금액'] != 0) & (estimated_sales['주말_매출_건수'] != 0 )]['주말_매출_금액'] / estimated_sales[(estimated_sales['주말_매출_금액'] != 0) & (estimated_sales['주말_매출_건수'] != 0) ]['주말_매출_건수']
estimated_sales['당월_건당_매출_금액'] = estimated_sales[(estimated_sales['당월_매출_금액'] != 0) & (estimated_sales['당월_매출_건수'] != 0 )]['당월_매출_금액'] / estimated_sales[(estimated_sales['당월_매출_금액'] != 0) & (estimated_sales['당월_매출_건수'] != 0 )]['당월_매출_건수']
estimated_sales['주중_건당_매출_금액'] = estimated_sales[(estimated_sales['주중_매출_금액'] != 0) & (estimated_sales['주중_매출_건수'] != 0 )]['주중_매출_금액'] / estimated_sales[(estimated_sales['주중_매출_금액'] != 0) & (estimated_sales['주중_매출_건수'] != 0 )]['주중_매출_건수']

estimated_sales['시간대_21~06_건당_매출_금액'] =  estimated_sales[(estimated_sales['시간대_21~06_매출_금액'] != 0) & (estimated_sales['시간대_21~06_매출_건수'] != 0) ]['시간대_21~06_매출_금액'] / estimated_sales[(estimated_sales['시간대_21~06_매출_금액'] != 0) & (estimated_sales['시간대_21~06_매출_건수'] != 0 )]['시간대_21~06_매출_건수']
estimated_sales['시간대_06~11_건당_매출_금액'] =  estimated_sales[(estimated_sales['시간대_06~11_매출_금액'] != 0) & (estimated_sales['시간대_건수~11_매출_건수'] != 0) ]['시간대_06~11_매출_금액'] / estimated_sales[(estimated_sales['시간대_06~11_매출_금액'] != 0) & (estimated_sales['시간대_건수~11_매출_건수'] != 0) ]['시간대_건수~11_매출_건수']
estimated_sales['시간대_11~14_건당_매출_금액'] =  estimated_sales[(estimated_sales['시간대_11~14_매출_금액'] != 0) & (estimated_sales['시간대_건수~14_매출_건수'] != 0) ]['시간대_11~14_매출_금액'] / estimated_sales[(estimated_sales['시간대_11~14_매출_금액'] != 0) & (estimated_sales['시간대_건수~14_매출_건수'] != 0 )]['시간대_건수~14_매출_건수']
estimated_sales['시간대_14~17_건당_매출_금액'] =  estimated_sales[(estimated_sales['시간대_14~17_매출_금액'] != 0) & (estimated_sales['시간대_건수~17_매출_건수'] != 0) ]['시간대_14~17_매출_금액'] / estimated_sales[(estimated_sales['시간대_14~17_매출_금액'] != 0) & (estimated_sales['시간대_건수~17_매출_건수'] != 0) ]['시간대_건수~17_매출_건수']
estimated_sales['시간대_17~21_건당_매출_금액'] =  estimated_sales[(estimated_sales['시간대_17~21_매출_금액'] != 0) & (estimated_sales['시간대_건수~21_매출_건수'] != 0) ]['시간대_17~21_매출_금액'] / estimated_sales[(estimated_sales['시간대_17~21_매출_금액'] != 0) & (estimated_sales['시간대_건수~21_매출_건수'] != 0 )]['시간대_건수~21_매출_건수']

estimated_sales['남성_건당_매출_금액'] = estimated_sales[(estimated_sales['남성_매출_금액'] != 0) & (estimated_sales['남성_매출_건수'] != 0)]['남성_매출_금액'] / estimated_sales[(estimated_sales['남성_매출_금액'] != 0) & (estimated_sales['남성_매출_건수'] != 0)]['남성_매출_건수']
estimated_sales['여성_건당_매출_금액'] = estimated_sales[(estimated_sales['여성_매출_금액'] != 0) & (estimated_sales['여성_매출_건수'] != 0)]['여성_매출_금액'] / estimated_sales[(estimated_sales['여성_매출_금액'] != 0) & (estimated_sales['여성_매출_건수'] != 0)]['여성_매출_건수']

estimated_sales['연령대_10_건당_매출_금액'] = estimated_sales[(estimated_sales['연령대_10_매출_금액'] != 0) & (estimated_sales['연령대_10_매출_건수'] != 0)]['연령대_10_매출_금액'] / estimated_sales[(estimated_sales['연령대_10_매출_금액'] != 0) & (estimated_sales['연령대_10_매출_건수'] != 0)]['연령대_10_매출_건수']
estimated_sales['연령대_20_건당_매출_금액'] = estimated_sales[(estimated_sales['연령대_20_매출_금액'] != 0) & (estimated_sales['연령대_20_매출_건수'] != 0)]['연령대_20_매출_금액'] / estimated_sales[(estimated_sales['연령대_20_매출_금액'] != 0) & (estimated_sales['연령대_20_매출_건수'] != 0)]['연령대_20_매출_건수']
estimated_sales['연령대_30_건당_매출_금액'] = estimated_sales[(estimated_sales['연령대_30_매출_금액'] != 0) & (estimated_sales['연령대_30_매출_건수'] != 0)]['연령대_30_매출_금액'] / estimated_sales[(estimated_sales['연령대_30_매출_금액'] != 0) & (estimated_sales['연령대_30_매출_건수'] != 0)]['연령대_30_매출_건수']
estimated_sales['연령대_40_건당_매출_금액'] = estimated_sales[(estimated_sales['연령대_40_매출_금액'] != 0) & (estimated_sales['연령대_40_매출_건수'] != 0)]['연령대_40_매출_금액'] / estimated_sales[(estimated_sales['연령대_40_매출_금액'] != 0) & (estimated_sales['연령대_40_매출_건수'] != 0)]['연령대_40_매출_건수']
estimated_sales['연령대_50_건당_매출_금액'] = estimated_sales[(estimated_sales['연령대_50_매출_금액'] != 0) & (estimated_sales['연령대_50_매출_건수'] != 0)]['연령대_50_매출_금액'] / estimated_sales[(estimated_sales['연령대_50_매출_금액'] != 0) & (estimated_sales['연령대_50_매출_건수'] != 0)]['연령대_50_매출_건수']
estimated_sales['연령대_60_이상_건당_매출_금액'] = estimated_sales[(estimated_sales['연령대_60_이상_매출_금액'] != 0) & (estimated_sales['연령대_60_이상_매출_건수'] != 0)]['연령대_60_이상_매출_금액'] / estimated_sales[(estimated_sales['연령대_60_이상_매출_금액'] != 0) & (estimated_sales['연령대_60_이상_매출_건수'] != 0)]['연령대_60_이상_매출_건수']


estimated_sales = estimated_sales.drop(columns = ['시간대_00~06_매출_금액', '시간대_21~24_매출_금액', '시간대_건수~06_매출_건수', '시간대_건수~24_매출_건수'],
                                       axis = 1)

cols_1 = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명']

cols_2 = ['당월_매출_금액', '주중_매출_금액', '주말_매출_금액',
         '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~06_매출_금액',
         '남성_매출_금액', '여성_매출_금액', 
         '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']

cols_3 = ['당월_매출_건수', '주중_매출_건수', '주말_매출_건수',
         '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_21~06_매출_건수',
         '남성_매출_건수', '여성_매출_건수',
         '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수']

cols_4 = ['당월_건당_매출_금액', '주중_건당_매출_금액', '주말_건당_매출_금액',
         '시간대_06~11_건당_매출_금액', '시간대_11~14_건당_매출_금액', '시간대_14~17_건당_매출_금액', '시간대_17~21_건당_매출_금액', '시간대_21~06_건당_매출_금액',
         '남성_건당_매출_금액', '여성_건당_매출_금액',
         '연령대_10_건당_매출_금액', '연령대_20_건당_매출_금액', '연령대_30_건당_매출_금액', '연령대_40_건당_매출_금액', '연령대_50_건당_매출_금액', '연령대_60_이상_건당_매출_금액']

cols = [*cols_1, *cols_2, *cols_3, *cols_4]

estimated_sales = estimated_sales[cols].rename(columns = {'시간대_건수~11_매출_건수' : '시간대_06~11_매출_건수',
                                                         '시간대_건수~14_매출_건수' : '시간대_11~14_매출_건수',
                                                         '시간대_건수~17_매출_건수' : '시간대_14~17_매출_건수',
                                                         '시간대_건수~21_매출_건수' : '시간대_17~21_매출_건수'})

estimated_sales = estimated_sales.fillna(0)

estimated_sales_year = estimated_sales.drop(columns = ['기준_년분기_코드']).groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명']).mean().reset_index()


In [185]:
estimated_sales_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22260 entries, 0 to 22259
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   상권_구분_코드_명          22260 non-null  object 
 1   상권_코드_명             22260 non-null  object 
 2   서비스_업종_코드_명         22260 non-null  object 
 3   당월_매출_금액            22260 non-null  float64
 4   주중_매출_금액            22260 non-null  float64
 5   주말_매출_금액            22260 non-null  float64
 6   시간대_06~11_매출_금액     22260 non-null  float64
 7   시간대_11~14_매출_금액     22260 non-null  float64
 8   시간대_14~17_매출_금액     22260 non-null  float64
 9   시간대_17~21_매출_금액     22260 non-null  float64
 10  시간대_21~06_매출_금액     22260 non-null  float64
 11  남성_매출_금액            22260 non-null  float64
 12  여성_매출_금액            22260 non-null  float64
 13  연령대_10_매출_금액        22260 non-null  float64
 14  연령대_20_매출_금액        22260 non-null  float64
 15  연령대_30_매출_금액        22260 non-null  float64
 16  연령대_

In [186]:
estimated_sales_year.describe()

,당월_매출_금액,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~06_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,당월_매출_건수,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,시간대_21~06_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,당월_건당_매출_금액,주중_건당_매출_금액,주말_건당_매출_금액,시간대_06~11_건당_매출_금액,시간대_11~14_건당_매출_금액,시간대_14~17_건당_매출_금액,시간대_17~21_건당_매출_금액,시간대_21~06_건당_매출_금액,남성_건당_매출_금액,여성_건당_매출_금액,연령대_10_건당_매출_금액,연령대_20_건당_매출_금액,연령대_30_건당_매출_금액,연령대_40_건당_매출_금액,연령대_50_건당_매출_금액,연령대_60_이상_건당_매출_금액
count,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000,22260.000
mean,1062640946.145,802511530.871,260129374.167,111129079.712,272294616.841,267047862.515,291642266.729,120527048.844,481871923.739,460040573.557,5467947.231,128612255.443,195192972.247,215418914.351,207079338.630,190138981.348,37290.151,28070.490,9219.661,5154.605,9698.510,7907.281,9946.395,4583.360,18410.666,17286.114,550.815,7512.541,8643.449,7110.534,6428.635,5450.804,120028.774,117526.518,76713.968,73303.281,91086.894,99379.961,94500.322,61651.246,104584.381,99875.949,12795.772,47897.835,75764.080,93815.128,99186.611,88032.473
std,9047572371.516,6244153815.244,3368505477.892,1128793053.329,2391244476.219,3012217121.365,2658313708.934,966641675.018,4291588415.044,3501184008.179,30194918.789,868653546.971,1646974501.834,1779184049.484,1853077935.384,1923202760.177,163067.984,114363.491,55517.077,28337.629,44538.901,42691.772,46089.280,23187.608,80845.420,80541.516,3427.287,40589.475,41150.259,31049.648,27671.458,34532.896,365870.181,351483.376,199886.993,347215.694,264550.525,311464.803,262679.870,156797.084,342942.819,272527.484,31256.943,141212.558,220725.240,277819.273,291938.368,358682.394
min,12.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,44760396.500,33318733.312,7264254.438,455157.062,5515970.938,7730705.125,11380580.000,0.000,18152589.812,17048966.688,0.000,1540946.938,4992326.750,7780006.750,8230405.688,4361182.438,755.438,573.750,135.938,9.000,96.000,152.750,197.750,0.000,339.750,301.250,0.000,40.250,103.250,152.000,145.438,89.938,15531.845,15175.875,12196.449,6023.906,11210.188,12987.338,12801.802,0.000,13720.012,13388.226,0.000,7162.937,10096.549,12365.150,13268.659,12362.934
50%,173930874.000,130800509.875,35388432.625,6353301.417,32719951.500,34624096.583,47264065.500,4074953.875,77802334.000,70572784.000,252146.000,11730044.000,25152070.625,34343474.458,34270478.250,21043269.375,4181.375,3104.625,937.000,151.125,882.000,888.000,1180.250,70.000,2045.625,1779.250,13.500,399.000,713.250,829.625,843.875,562.750,34702.491,33767.451,29220.623,16511.275,24443.959,28113.998,30947.468,15011.499,29215.157,31364.274,5043.738,17316.525,24694.714,29071.072,30150.050,27987.247
75%,644813194.875,490718901.312,141152221.062,46773312.875,145192586.375,135620289.688,175784241.438,47168537.208,295376391.625,272829959.938,2204467.000,59492252.188,107105896.812,129014912.750,124751192.812,92793427.750,23533.875,17600.250,5373.000,1834.000,5893.312,4834.188,6458.625,1351.188,11045.688,10647.750,161.562,3045.625,4457.125,4465.562,4356.250,3252.250,101181.101,100381.397,67113.209,47557.929,68541.247,80248.481,79361.747,58437.307,82430.210,82130.886,14102.683,40

## 임대료 전처리

In [187]:
rent = rent.rename(columns = {"행정동코드명" : "행정동_코드_명"}).drop(columns = ['행정동코드', '임대시세층구분코드', '기준년코드', '기준분기코드', '임대건수', '환산임대료평균'])[rent['임대시세층구분명'] != '전체층']

In [188]:
rent.head()

,행정동_코드_명,임대시세층구분명,보증금평균,월임대료평균
0,내곡동,1층,3286492,124345
1,내곡동,1층외,17778757,110169
3,방배3동,1층외,3590641,92720
5,방배3동,1층,1093228,94289
6,삼성2동,1층외,1312205,164525


In [189]:
rent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 849 entries, 0 to 1273
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   행정동_코드_명  849 non-null    object
 1   임대시세층구분명  849 non-null    object
 2   보증금평균     849 non-null    int64 
 3   월임대료평균    849 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 33.2+ KB


## 데이터 병합

In [190]:
df = pd.merge(living_popul_year, resident_popul_year)
df = pd.merge(df, area)
df = pd.merge(df, store_year)
df = pd.merge(df, working_popul_year)
df = pd.merge(df, facilitie_year)
df = pd.merge(df, estimated_sales_year)
df = pd.merge(df, rent)

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26970 entries, 0 to 26969
Columns: 114 entries, 상권_구분_코드_명 to 월임대료평균
dtypes: float64(83), int64(25), object(6)
memory usage: 23.5+ MB


In [192]:
df.head()

,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수,엑스좌표_값,와이좌표_값,자치구_코드_명,행정동_코드_명,영역_면적,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,총_직장_인구_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,중학교_수,고등학교_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수,당월_매출_금액,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~06_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,당월_매출_건수,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,시간대_21~06_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,당월_건당_매출_금액,주중_건당_매출_금액,주말_건당_매출_금액,시간대_06~11_건당_매출_금액,시간대_11~14_건당_매출_금액,시간대_14~17_건당_매출_금액,시간대_17~21_건당_매출_금액,시간대_21~06_건당_매출_금액,남성_건당_매출_금액,여성_건당_매출_금액,연령대_10_건당_매출_금액,연령대_20_건당_매출_금액,연령대_30_건당_매출_금액,연령대_40_건당_매출_금액,연령대_50_건당_매출_금액,연령대_60_이상_건당_매출_금액,임대시세층구분명,보증금평균,월임대료평균
0,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,107811.500,113781.750,121303.500,233529.500,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,73,57,62,93,95,216,62,64,72,97,95,261,201112,461090,강북구,우이동,60794,한식음식점,23,24,8,2,4,1,1,96,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,1739897842.000,1033559973.000,706337869.000,34758423.750,481068145.750,310010178.000,724466602.750,189594491.750,989044375.250,651314594.250,5154289.250,116807695.250,212122359.000,294670997.500,521347174.250,490256455.000,51956.500,33236.750,18719.750,1961.250,17455.500,10006.500,18257.750,4275.500,29305.250,21307.250,335.500,5561.500,6832.000,9084.250,14829.250,13972.000,33572.889,31134.794,37937.946,17773.801,27381.008,31010.383,39874.189,44545.446,33800.232,30709.071,15353.105,21074.157,31275.325,32766.235,35019.042,35398.899,1층외,814856,35822
1,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,107811.500,113781.750,121303.500,233529.500,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,73,57,62,93,95,216,62,64,72,97,95,261,201112,461090,강북구,우이동,60794,한식음식점,23,24,8,2,4,1,1,96,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,1739897842.000,1033559973.000,706337869.000,34758423.750,481068145.750,310010178.000,724466602.750,189594491.750,989044375.250,651314594.250,5154289.250,116807695.250,212122359.000,294670997.500,521347174.250,490256455.000,51956.500,33236.750,18719.750,1961.250,17455.500,10006.500,18257.750,4275.500,29305.250,21307.250,335.500,5561.500,6832.000,9084.250,14829.250,13972.000,33572.889,31134.794,37937.946,17773.801,27381.008,31010.383,39874.189,44545.446,33800.232,30709.071,15353.105,21074.157,31275.325,32766.235,35019.042,35398.899,1층,1600781,74305
2,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,107811.500,113781.750,121303.500,233529.500,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,73,57,62,93,95,216,62,64,72,97,95,261,201112,461090,강북구,우이동,60794,일식음식점,3,3,0,0,0,0,0,96,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,18435727.000,15543098.250,2892628.750,0.000,9658447.500,3074001.750,5623422.500,79855.250,7005386.750,11430340.250,561557.250,6405609.500,2608330.250,3453367.250,3140322.750,2266540.000,1519.250,1359.250,160.000,0.000,821.500,247.000,441.750,9.000,463.250,1056.000,55.250,601.250,198.500,296.750,264.000,103.500,12216.994,1151